In [1]:
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True) #--> not working

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# TASK 2: Clone & Explore dataset

In [3]:
#clone the diabetes dataset from the github repository

! git clone https://github.com/education454/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (6/6), 13.02 KiB | 1.45 MiB/s, done.


In [5]:
#check if the dataset exists
! ls diabetes_dataset/

diabetes.csv  new_test.csv


In [3]:
#create spark dataframe
df = spark.read.csv(f"gs://dataproc-staging-us-central1-139389956614-ujvfk5z1/notebooks/jupyter/diabetes.csv", header=True, inferSchema=True)

In [4]:
#display the dataframe

df.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



In [5]:
#print the schema
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [6]:
#count the total no. of diabetic and non-diabetic class
df.groupBy('outcome').count().show()

+-------+-----+
|outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [8]:
#get the summary statistics

df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

# TASK 3: Data Cleaning & Preparation

In [21]:
[x for x in df.columns]

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [19]:
#check for null values

for col in df.columns:
    print(col, " : ", df[df[col].isNull()].count() )

Pregnancies  :  0
Glucose  :  0
BloodPressure  :  0
SkinThickness  :  0
Insulin  :  0
BMI  :  0
DiabetesPedigreeFunction  :  0
Age  :  0
Outcome  :  0


In [25]:
#look for the unnecessary values present
columns_list = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

def count_zeros():
    for i in columns_list:
        print(i, " : ", df[df[i]==0].count())


In [26]:
count_zeros()

Glucose  :  13
BloodPressure  :  90
SkinThickness  :  573
Insulin  :  956
BMI  :  28


In [43]:
df.agg({'glucose':'mean'}).first()[0]   #head()[0]   #collect()[0][0]

121.1825

In [44]:
#calculate and replace the unnecessary values by the mean value

from pyspark.sql.functions import *

for i in df.columns[1:6]:
    data = df.agg({i:'mean'}).first()[0]
    print("mean value for {0} is {1}".format(i, data))
    
    df = df.withColumn(i, when(df[i]==0, int(data)).otherwise(df[i]))

mean value for Glucose is 121.1825
mean value for BloodPressure is 69.1455
mean value for SkinThickness is 20.935
mean value for Insulin is 80.254
mean value for BMI is 32.192999999999984


In [46]:
#display the dataframe 
df.show(10,0)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI |DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|2          |138    |62           |35           |80     |33.6|0.127                   |47 |1      |
|0          |84     |82           |31           |125    |38.2|0.233                   |23 |0      |
|0          |145    |69           |20           |80     |44.2|0.63                    |31 |1      |
|0          |135    |68           |42           |250    |42.3|0.365                   |24 |1      |
|1          |139    |62           |41           |480    |40.7|0.536                   |21 |0      |
|0          |173    |78           |32           |265    |46.5|1.159                   |58 |0      |
|4          |99     |72           |17           |80     |25.6|0.294                   |28 |0      |


# TASK 4: Correlation Analysis & Feature Selection

In [47]:
#find the correlation among the set of input & output variables

for i in df.columns:
    print("corelation outcome for {0} is {1}".format(i, df.stat.corr('Outcome', i)))

corelation outcome for Pregnancies is 0.22443699263363961
corelation outcome for Glucose is 0.48796646527321064
corelation outcome for BloodPressure is 0.17171333286446713
corelation outcome for SkinThickness is 0.1659010662889893
corelation outcome for Insulin is 0.1711763270226193
corelation outcome for BMI is 0.2827927569760082
corelation outcome for DiabetesPedigreeFunction is 0.1554590791569403
corelation outcome for Age is 0.23650924717620253
corelation outcome for Outcome is 1.0


In [50]:
#feature selection

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'], outputCol='feature')

output_data = assembler.transform(df)

In [53]:
#print the schema
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- feature: vector (nullable = true)



In [54]:
#display dataframe

output_data.show(5,0)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+-------------------------------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI |DiabetesPedigreeFunction|Age|Outcome|feature                                    |
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+-------------------------------------------+
|2          |138    |62           |35           |80     |33.6|0.127                   |47 |1      |[2.0,138.0,62.0,35.0,80.0,33.6,0.127,47.0] |
|0          |84     |82           |31           |125    |38.2|0.233                   |23 |0      |[0.0,84.0,82.0,31.0,125.0,38.2,0.233,23.0] |
|0          |145    |69           |20           |80     |44.2|0.63                    |31 |1      |[0.0,145.0,69.0,20.0,80.0,44.2,0.63,31.0]  |
|0          |135    |68           |42           |250    |42.3|0.365                   |24 |1      |[0.0,135.0,68.0,42.0,250.0,42.3,0.365

# TASK 5: Split Dataset & Build the Model

In [ ]:
#create final data

In [ ]:
#print schema of final data

In [ ]:
#split the dataset ; build the model

In [ ]:
#summary of the model

# TASK 6: Evaluate and Save the Model

In [ ]:
# save model

In [ ]:
# load saved model back to the environment

# TASK 7: Prediction on New Data with the saved model

In [ ]:
#create a new spark dataframe

In [ ]:
#print the schema

In [ ]:
#create an additional feature merged column 

In [ ]:
#print the schema

In [ ]:
#use model to make predictions

In [ ]:
#display the predictions